## Clustering을 통해 분류모델의 target을 만들어주기

In [1]:
#!pip install kmodes

In [1]:
import pandas as pd
import pickle
from kmodes.kmodes import KModes
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('wine.csv', index_col=0)

print('총 데이터 크기 :', df.shape)    #(323065, 16)
df.head()

총 데이터 크기 : (323065, 16)


,wine,winery,category,designation,varietal,appellation,alcohol,price,rating,reviewer,review,country,region_1,region_2,region_3,region_4
0,J. Lohr 2000 Hilltop Vineyard Cabernet Sauvign...,J. Lohr,Red,Hilltop Vineyard,Cabernet Sauvignon,"Paso Robles, Central Coast, California, US",13.5,32.0,87,NaN,"Held back nearly five years, this Cab still ha...",US,California,Central Coast,Paso Robles,NaN
1,Antucura 2010 Pinot Noir (Vista Flores),Antucura,Red,NaN,Pinot Noir,"Vista Flores, Mendoza Province, Argentina",15.0,17.0,85,Michael Schachner,"Rubbery, bold and earthy on the nose, then ful...",Argentina,Mendoza Province,Vista Flores,NaN,NaN
2,Quinta do Portal 1999 Quinta do Portal Reserva...,Quinta do Portal,Red,Quinta do Portal Reserva,"Red Blends, Red Blends","Douro, Portugal",13.5,24.0,90,Roger Voss,"Smoky, meaty aromas lead into perfumed fruit f...",Portugal,Douro,NaN,NaN,NaN
3,Tenuta di Ghizzano 2006 Il Ghizzano Red (Toscana),Tenuta di Ghizzano,Red,Il Ghizzano,"Red Blends, Red Blends","Toscana, Tuscany, Italy",13.5,18.0,86,NaN,The nose is rather neutral save for distant ar...,Italy,Tuscany,Toscana,NaN,NaN
4,Tenuta San Francesco 2007 Tramonti White (Camp...,Tenuta San Francesco,White,Tramonti,White Blend,"Campania, Southern Italy, Italy",13.5,21.0,85,NaN,"This intriguing blend of Falanghina, Biancolel...",Italy,Southern Italy,Campania,NaN,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 323065 entries, 0 to 323203
Data columns (total 16 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   wine         323065 non-null  object 
 1   winery       323065 non-null  object 
 2   category     323065 non-null  object 
 3   designation  229570 non-null  object 
 4   varietal     302027 non-null  object 
 5   appellation  320458 non-null  object 
 6   alcohol      323065 non-null  float64
 7   price        323065 non-null  float64
 8   rating       323065 non-null  int64  
 9   reviewer     250924 non-null  object 
 10  review       321937 non-null  object 
 11  country      320458 non-null  object 
 12  region_1     320413 non-null  object 
 13  region_2     272146 non-null  object 
 14  region_3     132104 non-null  object 
 15  region_4     59 non-null      object 
dtypes: float64(2), int64(1), object(13)
memory usage: 41.9+ MB


In [3]:
df.fillna('NaN', inplace=True)

## 클러스터링에 사용할 feature만 골라내기
+ wine, appellation, rating, reviewer, review => 클러스터링에는 사용 x, 이후 추천 시스템에서 사용

In [6]:
df.columns

Index(['wine', 'winery', 'category', 'designation', 'varietal', 'appellation',
       'alcohol', 'price', 'rating', 'reviewer', 'review', 'country',
       'region_1', 'region_2', 'region_3', 'region_4'],
      dtype='object')

In [4]:
df_for_clustering = df.iloc[:, [1,2,3,4,6,7,11,12,13,14,15]]
print(df_for_clustering.shape)    #(323065, 11)
df_for_clustering.head()

(323065, 11)


,winery,category,designation,varietal,alcohol,price,country,region_1,region_2,region_3,region_4
0,J. Lohr,Red,Hilltop Vineyard,Cabernet Sauvignon,13.5,32.0,US,California,Central Coast,Paso Robles,NaN
1,Antucura,Red,NaN,Pinot Noir,15.0,17.0,Argentina,Mendoza Province,Vista Flores,NaN,NaN
2,Quinta do Portal,Red,Quinta do Portal Reserva,"Red Blends, Red Blends",13.5,24.0,Portugal,Douro,NaN,NaN,NaN
3,Tenuta di Ghizzano,Red,Il Ghizzano,"Red Blends, Red Blends",13.5,18.0,Italy,Tuscany,Toscana,NaN,NaN
4,Tenuta San Francesco,White,Tramonti,White Blend,13.5,21.0,Italy,Southern Italy,Campania,NaN,NaN


In [8]:
df_for_clustering.to_csv('df_for_clustering.csv')
df_for_clustering.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 323065 entries, 0 to 323203
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   winery       323065 non-null  object 
 1   category     323065 non-null  object 
 2   designation  323065 non-null  object 
 3   varietal     323065 non-null  object 
 4   alcohol      323065 non-null  float64
 5   price        323065 non-null  float64
 6   country      323065 non-null  object 
 7   region_1     323065 non-null  object 
 8   region_2     323065 non-null  object 
 9   region_3     323065 non-null  object 
 10  region_4     323065 non-null  object 
dtypes: float64(2), object(9)
memory usage: 29.6+ MB


In [ ]:
#n_clusters 에 따른 cost 확인
#cost = []
#k = [5, 10, 30, 50, 100, 300, 500, 1000]
#for num_clusters in k:
#    kmode = KModes(n_clusters=num_clusters, n_jobs=8, init='random', n_init=5, verbose=1)
#    kmode.fit_predict(df_for_clustering)
#    cost.append(kmode.cost_)
    
#plt.plot(k, cost, 'bx-')
#plt.xlabel('No. of clusters')
#plt.ylabel('Cost')
#plt.title('Elbow Method For Optimal k')
#plt.show()

## n_clusters=100

In [6]:
kmode_100 = KModes(n_clusters=100, n_jobs=8, init='random', n_init=5, verbose=1)
clusters_100 = kmode_100.fit_predict(df_for_clustering)

cluster_df = df[['wine', 'winery', 'category', 'designation', 'varietal',
    'appellation', 'alcohol', 'price', 'rating', 'reviewer', 'review']]

cluster_df.insert(0, 'cluster_100', clusters_100, True)

cluster_df.cluster_100.value_counts()

Best run was number 2


3     17334
15    14163
5     12992
14     9655
12     9355
      ...  
62      825
50      719
86      705
66      509
64      502
Name: cluster_100, Length: 100, dtype: int64

In [45]:
cluster_df.reset_index(inplace=True, drop=True)

cluster_df 

,cluster_100,wine,winery,category,designation,varietal,appellation,alcohol,price,rating,reviewer,review
0,3,J. Lohr 2000 Hilltop Vineyard Cabernet Sauvign...,J. Lohr,Red,Hilltop Vineyard,Cabernet Sauvignon,"Paso Robles, Central Coast, California, US",13.5,32.0,87,NaN,"Held back nearly five years, this Cab still ha..."
1,53,Antucura 2010 Pinot Noir (Vista Flores),Antucura,Red,NaN,Pinot Noir,"Vista Flores, Mendoza Province, Argentina",15.0,17.0,85,Michael Schachner,"Rubbery, bold and earthy on the nose, then ful..."
2,61,Quinta do Portal 1999 Quinta do Portal Reserva...,Quinta do Portal,Red,Quinta do Portal Reserva,"Red Blends, Red Blends","Douro, Portugal",13.5,24.0,90,Roger Voss,"Smoky, meaty aromas lead into perfumed fruit f..."
3,52,Tenuta di Ghizzano 2006 Il Ghizzano Red (Toscana),Tenuta di Ghizzano,Red,Il Ghizzano,"Red Blends, Red Blends","Toscana, Tuscany, Italy",13.5,18.0,86,NaN,The nose is rather neutral save for distant ar...
4,10,Tenuta San Francesco 2007 Tramonti White (Camp...,Tenuta San Francesco,White,Tramonti,White Blend,"Campania, Southern Italy, Italy",13.5,21.0,85,NaN,"This intriguing blend of Falanghina, Biancolel..."
...,...,...,...,...,...,...,...,...,...,...,...,...
323060,3,Maddalena 2017 Rosé (Paso Robles),Maddalena,Rose,NaN,Rosé,"Paso Robles, Central Coast, California, US",12.5,18.0,87,Matt Kettmann,"Candied raspberry, cranberry, orange juice and..."
323061,10,Toscolo 2015 Vernaccia di San Gimignano,Toscolo,White,NaN,"Vernaccia, Italian White","Vernaccia di San Gimignano, Tuscany, Italy",12.5,11.0,87,Kerin O’Keefe,"Aromas of white spring flower, yellow pear and..."
323062,65,Domaine G. Metz 2017 Pinot Blanc (Alsace),Domaine G. Metz,White,NaN,Pinot Blanc,"Alsace, Alsace, France",13.0,20.0,90,Anne Krebiehl MW,"A tinge of earth clings to the ripe, almost ju..."
323063,1,Huston 2019 Chicken Dinner Red (Idaho),Huston,Red,Chicken Dinner,"Red Blends, Red Blends","Idaho, Idaho, US",14.3,18.0,87,Sean P. Sullivan,"Fruit-forward aromas of plum, huckleberry and ..."


In [10]:
cluster_df.to_csv('wine_cluster_100.csv')

In [ ]:
with open('model_100clusters.pkl','wb') as pickle_file:    
    pickle.dump(kmode_100, pickle_file)   

## n_clusters=300

In [5]:
kmode_300 = KModes(n_clusters=100, n_jobs=8, init='random', n_init=5, verbose=1)
clusters_300 = kmode_300.fit_predict(df_for_clustering)

cluster_df = df[['wine', 'winery', 'category', 'designation', 'varietal',
    'appellation', 'alcohol', 'price', 'rating', 'reviewer', 'review']]

cluster_df.insert(0, 'cluster_300', clusters_300, True)

cluster_df.cluster_300.value_counts()

Best run was number 5


6     13333
0     12190
13    10338
7      8218
2      7661
      ...  
50      483
91      426
75      389
43      378
94      325
Name: cluster_300, Length: 100, dtype: int64

In [6]:
cluster_df.reset_index(inplace=True, drop=True)

cluster_df 

,cluster_300,wine,winery,category,designation,varietal,appellation,alcohol,price,rating,reviewer,review
0,51,J. Lohr 2000 Hilltop Vineyard Cabernet Sauvign...,J. Lohr,Red,Hilltop Vineyard,Cabernet Sauvignon,"Paso Robles, Central Coast, California, US",13.5,32.0,87,NaN,"Held back nearly five years, this Cab still ha..."
1,53,Antucura 2010 Pinot Noir (Vista Flores),Antucura,Red,NaN,Pinot Noir,"Vista Flores, Mendoza Province, Argentina",15.0,17.0,85,Michael Schachner,"Rubbery, bold and earthy on the nose, then ful..."
2,36,Quinta do Portal 1999 Quinta do Portal Reserva...,Quinta do Portal,Red,Quinta do Portal Reserva,"Red Blends, Red Blends","Douro, Portugal",13.5,24.0,90,Roger Voss,"Smoky, meaty aromas lead into perfumed fruit f..."
3,7,Tenuta di Ghizzano 2006 Il Ghizzano Red (Toscana),Tenuta di Ghizzano,Red,Il Ghizzano,"Red Blends, Red Blends","Toscana, Tuscany, Italy",13.5,18.0,86,NaN,The nose is rather neutral save for distant ar...
4,9,Tenuta San Francesco 2007 Tramonti White (Camp...,Tenuta San Francesco,White,Tramonti,White Blend,"Campania, Southern Italy, Italy",13.5,21.0,85,NaN,"This intriguing blend of Falanghina, Biancolel..."
...,...,...,...,...,...,...,...,...,...,...,...,...
323060,77,Maddalena 2017 Rosé (Paso Robles),Maddalena,Rose,NaN,Rosé,"Paso Robles, Central Coast, California, US",12.5,18.0,87,Matt Kettmann,"Candied raspberry, cranberry, orange juice and..."
323061,99,Toscolo 2015 Vernaccia di San Gimignano,Toscolo,White,NaN,"Vernaccia, Italian White","Vernaccia di San Gimignano, Tuscany, Italy",12.5,11.0,87,Kerin O’Keefe,"Aromas of white spring flower, yellow pear and..."
323062,30,Domaine G. Metz 2017 Pinot Blanc (Alsace),Domaine G. Metz,White,NaN,Pinot Blanc,"Alsace, Alsace, France",13.0,20.0,90,Anne Krebiehl MW,"A tinge of earth clings to the ripe, almost ju..."
323063,7,Huston 2019 Chicken Dinner Red (Idaho),Huston,Red,Chicken Dinner,"Red Blends, Red Blends","Idaho, Idaho, US",14.3,18.0,87,Sean P. Sullivan,"Fruit-forward aromas of plum, huckleberry and ..."


In [7]:
cluster_df.to_csv('wine_cluster_300.csv')

In [ ]:
with open('model_300clusters.pkl','wb') as pickle_file:    
    pickle.dump(kmode_300, pickle_file)  